# FMRI DATA ANALYSIS - NARPS DATASET BY TEAM V55J

This notebook reproduce the analysis made by team **V55J** for the NARPS experiment (Botvinik-Nezer & al, 2020). 


**To use this notebook :** 
- Download fMRIprep data available [**here**](https://openneuro.org/datasets/ds001734/versions/1.0.4) or in [**datalad**](http://datasets.datalad.org/?dir=/workshops/nih-2017/ds001734).
- Modify the paths:
    - **exp_dir**: directory where the ds001734-download repository is stored
    - **result_dir**: directory where the intermediate and final repositories will be stored
    - **working_dir**: name of the directory where intermediate results will be stored
    - **output_dir**: name of the directory where final results will be stored

Botvinik-Nezer, R., Holzmeister, F., Camerer, C.F., Dreber, A., Huber, J., Johannesson, M., Kirchler, M., Iwanir, R., Mumford, J.A., ..., Nichols, T.E., Poldrack, R.A., Schonberg, T. (2020). Variability in the analysis of a single neuroimaging dataset by many teams. Nature. https://doi.org/10.1038/s41586-020-2314-9.

In [ ]:
# Set the path to matlab 
## NECESSARY TO USE SPM
from nipype.interfaces import spm
matlab_cmd = '/opt/spm12-r7771/run_spm12.sh /opt/matlabmcr-2010a/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

In [ ]:
from narps_open.pipelines import team_V55J

from os.path import join as opj
import os
import json

In [ ]:
team_ID = "V55J"
# Important directories
## exp_dir : where the data are stored (where the ds001734 directory is stored)
exp_dir = '../data/original'

## result_dir : where the intermediate and final results will be store
result_dir = "../data/derived/reproduced"

## working_dir : where the intermediate outputs will be store
working_dir = f"NARPS-{team_ID}-reproduced/intermediate_results"

## output_dir : where the final results will be store
output_dir = f"NARPS-{team_ID}-reproduced"

In [ ]:
dir_list = os.listdir(exp_dir)

In [ ]:
# Subject list (to which we will do the analysis)
subject_list = []
for dirs in dir_list:
    if dirs[0:3] == 'sub':
        subject_list.append(dirs[-3:])

In [ ]:
n_sub = len(subject_list)
# Run to use for this analysis
run_list = ['01', '02', '03', '04']

#TR
with open(opj(exp_dir, 'task-MGT_bold.json'), 'rt') as fp:
    task_info = json.load(fp)
TR = task_info['RepetitionTime']

In [ ]:
#FWHM to smooth (team chose a kernel of 6mm for smoothing)
fwhm = 6

contrast_list = ['01', '02']

In [ ]:
l2_analysis_equal_indiff = team_V55J.get_l2_analysis(subject_list, n_sub, contrast_list, "equalIndifference", exp_dir, result_dir, working_dir, output_dir)

l2_analysis_equal_indiff.run('MultiProc', plugin_args={'n_procs': 16})

In [ ]:
l2_analysis_equal_range = team_V55J.get_l2_analysis(subject_list, n_sub, contrast_list, "equalRange", exp_dir, result_dir, working_dir, output_dir)

l2_analysis_equal_range.run('MultiProc', plugin_args={'n_procs': 16})

In [ ]:
contrast_list = ['02']
method = 'groupComp'

l2_analysis_groupcomp = team_V55J.get_l2_analysis(subject_list, n_sub, contrast_list, "groupComp", 
                  exp_dir, result_dir, working_dir, output_dir)


l2_analysis_groupcomp.run('MultiProc', plugin_args={'n_procs': 16})

In [ ]:
team_V55J.reorganize_results(result_dir, output_dir, n_sub, team_ID)